Personality Classifications

SIMPING

Openness to experience (inventive/curious vs. consistent/cautious)

Conscientiousness (efficient/organized vs. easy-going/careless)

Extraversion (outgoing/energetic vs. solitary/reserved)

Agreeableness (friendly/compassionate vs. challenging/detached)

Neuroticism (sensitive/nervous vs. secure/confident)

Characteristics of Songs We Will Be Looking At

Tempo

Danceability

Energy 

Instrumentalness



In [2]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

TensorFlow 2.x selected.


In [0]:
col_names = ['Danceability', 'Tempo', 'Energy', 'Instrumentalness', 'Type']
personality_types = ['O','C', 'E', 'A', 'N']


In [0]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=col_names, header=0)
test = pd.read_csv(test_path, names=col_names, header=0)

In [36]:
training_label = train.pop('Type')
test_label = test.pop('Type')
train.head()

,Danceability,Tempo,Energy,Instrumentalness
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [0]:
#define input function for model
def input_fn(features, labels, training=True, batch_size=256):
    #Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)


In [39]:
#Feature columns
feat_cols = []
for k in train.keys():
    feat_cols.append(tf.feature_column.numeric_column(key=k))
print(feat_cols)


[NumericColumn(key='Danceability', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Tempo', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Energy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Instrumentalness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


At this point we can choose from 2 different classifiers. A DNN(Deep Neural Network) or a linear classifier. 

Both are shown below


In [40]:
#DNN below
classifier = tf.estimator.DNNClassifier(
    hidden_units=[30,10],
    feature_columns=feat_cols,
    n_classes=3,
)
#n_classes must be equal to 5 for personality classifier


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmph37hbghr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [41]:
classifier.train(input_fn=lambda: input_fn(train,training_label, training=True), steps= 5000)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

In [42]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test,test_label, training=False))

print('\n Test accuracy : {accuracy:0.3f}\n'.format(**eval_result))
#Test accuract : 0.967 for testing data

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-25T15:01:00Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmph37hbghr/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22061s
INFO:tensorflow:Finished evaluation at 2020-03-25-15:01:00
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, averag

ERROR! Session/line number was not unique in database. History logging moved to new session 59


NameError: ignored